In [ ]:
import torch
import numpy as np
from tqdm import tqdm

data_name, target_dim = "brainq_mat", 1
data_file = "../data/" + data_name + ".npy"
_mat = np.load(data_file)
batch_size = 2**21

device = torch.device("cuda")
num_node = _mat.shape[target_dim]
seg_len = _mat.shape[1-target_dim]
adj_mat = np.zeros((num_node, num_node))
for i in tqdm(range(num_node)):
    if target_dim == 0: slice1 = _mat[i,:]
    else: slice1 = _mat[:,i]
    slice1 = np.tile(slice1, num_node-i-1) 
    
    if target_dim == 0: slice2 = _mat[i+1:,:].flatten()
    else: slice2 = np.transpose(_mat[:,i+1:]).flatten()
    slice1 = torch.tensor(slice1, device=device)
    slice2 = torch.tensor(slice2, device=device)
    
    curr_len = seg_len*(num_node-i-1)
    #print(num_node)
    #print(_mat[:,i+1:].shape)
    #print(f'curr len:{curr_len}, slice2:{slice2.shape}')
    curr_adj_row = torch.zeros(num_node-i-1, device=device, dtype = torch.float64)    
    for j in range(0, curr_len, batch_size):
        if curr_len - j < batch_size: curr_batch_size = curr_len - j
        else: curr_batch_size = batch_size           
        curr_idx = torch.arange(j, j+curr_batch_size, device=device) // seg_len
        curr_val = torch.square(slice1[j:j+curr_batch_size] - slice2[j:j+curr_batch_size])        
        
        #print(f'row:{curr_adj_row.shape}, idx:{curr_idx.shape}, val:{curr_val.shape}')
        curr_adj_row.scatter_(0, curr_idx, curr_val, reduce='add')
    
    curr_adj_row = torch.sqrt(curr_adj_row)
    curr_adj_row = curr_adj_row.clone().cpu().numpy()
    adj_mat[i, i+1:] = curr_adj_row
    adj_mat[i+1:, i] = curr_adj_row

out_file = data_name + "_adj" + str(target_dim) + ".npy"
np.save(out_file, adj_mat)

  0%|                                                                                                                                                                | 0/21764 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
 59%|██████████████████████████████████████████████████████████████████████████████████████▊                                                             | 12769/21764 [11:58<03:34, 41.89it/s]